In [2]:
!pip install sentence-transformers

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
# import dependencies
import json
import torch
import faiss
from pprint import pprint
import numpy as np
from sentence_transformers import SentenceTransformer, util

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Load data and preprocess

In [4]:
# Load documents from JSON file
with open('data/data.json', 'r') as file:
    documents = json.load(file)

In [5]:
# Load a transformers model
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [6]:
x = embedder.encode(['spanish flu'], convert_to_tensor=True)
x.numpy().shape

(1, 768)

In [13]:
# Compute sentence embeddings for every text in the documents
corpus = [d['text'] for d in documents]
corpus[0]

'A pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century. The term was not used yet but was for later pandemics including the 1918 influenza pandemic (Spanish flu). Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.'

In [7]:
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [14]:
## move the tensor data to CPU for the FAISS index to work
corpus_embeddings = corpus_embeddings.cpu()
corpus_embeddings[0]

tensor([ 2.9952e-01, -7.0211e-01, -4.3361e-01, -1.9087e-02, -2.4387e-01,
         5.7081e-01,  8.7062e-02, -3.7161e-01, -1.4103e-01, -1.0819e+00,
         1.6249e-01, -3.7736e-01, -1.0932e+00,  3.5094e-01,  5.4796e-01,
         1.2615e-02,  7.2166e-01,  2.2545e-01, -4.9946e-01,  1.9949e-01,
        -4.7665e-01,  1.0053e-01,  2.1273e-01,  2.6754e-01, -1.4378e+00,
        -7.9652e-02,  2.4992e-01, -4.7607e-02,  1.0231e+00, -5.2525e-01,
         4.1967e-01,  3.4385e-01, -1.5538e-01, -9.8291e-01, -4.6316e-01,
        -1.3336e-01, -1.5561e-01,  7.5304e-01,  1.2626e-01,  1.3102e-01,
         2.9170e-02, -6.4067e-01, -6.7099e-02, -3.7497e-01,  1.3653e-01,
         4.2604e-01, -4.8826e-01,  6.3649e-01, -2.8571e-01,  2.3233e-01,
        -1.3344e-02, -9.8263e-01,  7.5536e-01,  1.2606e+00,  1.9006e-01,
         2.3629e-01, -5.4619e-01,  2.9539e-01, -5.9843e-01,  2.0578e-01,
        -3.1807e-01, -6.9939e-01, -4.4655e-01, -2.5983e-02,  3.0219e-01,
        -3.6411e-01, -6.5929e-01,  6.9335e-01,  6.5

In [15]:
corpus_embeddings[-1]

tensor([ 7.3829e-01, -3.7307e-01,  3.8709e-01,  4.6272e-01,  1.8671e-01,
        -5.3994e-01,  4.9802e-01, -7.1573e-02,  3.6149e-01, -5.1980e-02,
         7.1352e-01,  3.0133e-02, -1.2299e+00,  8.9091e-01,  1.4684e-01,
         7.7680e-02,  1.1512e-01,  3.9175e-01,  2.3867e-01, -1.6696e-01,
        -3.0103e-01,  8.3501e-02,  1.5501e-01,  1.6569e-01, -5.2334e-01,
        -7.6129e-01,  5.0878e-01,  2.4885e-01,  1.4092e+00, -5.9079e-01,
         2.4068e-01, -3.6413e-01, -2.4202e-02, -6.5462e-01, -1.0830e-01,
        -6.7298e-02, -4.0739e-01,  2.2640e-02, -3.2402e-01, -3.7856e-01,
        -1.5790e-01, -2.4392e-01, -1.1966e-01, -1.2017e-01, -1.9109e-01,
        -2.7295e-01,  6.1089e-01,  8.0647e-01,  6.0772e-02,  5.6018e-01,
         3.9073e-01, -7.6066e-01,  3.9909e-01, -1.4104e-01,  2.5068e-01,
        -1.0269e-01, -1.5311e-01, -1.0042e+00,  3.1872e-01,  3.3264e-01,
        -2.5117e-01, -7.8069e-01,  1.5139e-01,  3.4694e-02,  1.0009e+00,
        -2.9685e-01, -7.1686e-01,  4.0429e-01, -1.0

In [9]:
corpus_embeddings.numpy().shape

(26, 768)

### Create a FAISS index

In [10]:
# Create a flat Faiss index
inner_index = faiss.IndexFlatIP(768) # the size of our vector space
index = faiss.IndexIDMap(inner_index)

index.add_with_ids(corpus_embeddings.numpy(), 
                   np.array(range(0, len(corpus))))

# save the index for future use
faiss.write_index(index, 'data/pandemics')

### Search the documents

In [11]:
# Build a search function that finds the most relevant search results
def search(query, documents, k=5):
    # the difference from milestone1 is the use of embedder which is SBERT instead of BERT
    encoded_query = embedder.encode([query])
    top_k = index.search(encoded_query, k)
    scores = top_k[0][0]
    results = [documents[_id] for _id in top_k[1][0]]
    return list(zip(results, scores))
    

In [12]:
encoded_query = embedder.encode(["spanish flu casualties"])
encoded_query.shape

(1, 768)

In [ ]:
top_k = index.search(encoded_query, 2)
top_k

In [ ]:
pprint(search("spanish flu casualties", corpus, k=2))